In [1]:
import pandas as pd


In [2]:
interactions_df = pd.read_csv('dataset/RAW_interactions.csv')
recipes_df = pd.read_csv('dataset/RAW_recipes.csv')

In [3]:
# Display the first few rows of each dataset
print("Interactions Data:")
print(interactions_df.head())
print("\nInteractions Data Columns:")
print(interactions_df.columns)

Interactions Data:
   user_id  recipe_id        date  rating  \
0    38094      40893   2/17/2003       4   
1  1293707      40893  12/21/2011       5   
2     8937      44394   12/1/2002       4   
3   126440      85009   2/27/2010       5   
4    57222      85009   10/1/2011       5   

                                              review  
0  Great with a salad. Cooked on top of stove for...  
1  So simple, so delicious! Great for chilly fall...  
2  This worked very well and is EASY.  I used not...  
3  I made the Mexican topping and took it to bunk...  
4  Made the cheddar bacon topping, adding a sprin...  

Interactions Data Columns:
Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')


In [4]:
print("\nRecipes Data:")
print(recipes_df.head())


Recipes Data:
  arriba   baked winter squash mexican style  137739   55   47892   9/16/2005  \
0                  all in the kitchen  chili   31490   30   26278   6/17/2002   
1                         alouette  potatoes  112140  130  196586   2/25/2005   
2         amish  tomato ketchup  for canning   59389   45   68585   4/14/2003   
3                    apple a day  milk shake   44061  190   41706  10/25/2002   
4             backyard style  barbecued ribs   25274   15   21730   4/14/2002   

  ['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']  \
0  ['30-minutes-or-less', 'time-to-make', 'course...                                                                                                                                                                              

In [5]:
print("\nRecipes Data Columns:")
print(recipes_df.columns)


Recipes Data Columns:
Index(['arriba   baked winter squash mexican style', '137739', '55', '47892',
       '9/16/2005',
       '['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']',
       '[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]', '11',
       '['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash espec

In [6]:
# Load the data again, specifying header=None to prevent the first row from being interpreted as column names
recipes_df = pd.read_csv('dataset/RAW_recipes.csv', header=None)

In [7]:
# Rename the columns appropriately
recipes_df.columns = ['recipe_name', 'recipe_id', 'minutes', 'contributor_id', 'submitted', 'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']


In [8]:
print("\nRecipes Data Columns:")
print(recipes_df.columns)


Recipes Data Columns:
Index(['recipe_name', 'recipe_id', 'minutes', 'contributor_id', 'submitted',
       'tags', 'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')


In [9]:
# Convert the necessary columns to integers
recipes_df['recipe_id'] = recipes_df['recipe_id'].astype(int)

In [10]:
recipes_df.rename(columns={'id': 'recipe_id'}, inplace=True)

In [11]:
# Check for missing values
print("\nMissing values in interactions data:")
print(interactions_df.isnull().sum())

print("\nMissing values in recipes data:")
print(recipes_df.isnull().sum())


Missing values in interactions data:
user_id      0
recipe_id    0
date         0
rating       0
review       1
dtype: int64

Missing values in recipes data:
recipe_name       0
recipe_id         0
minutes           0
contributor_id    0
submitted         0
tags              0
nutrition         0
n_steps           0
steps             0
description       6
ingredients       0
n_ingredients     0
dtype: int64


In [12]:
# Drop rows with missing values in interactions data
interactions_df = interactions_df.dropna()

# Drop rows with missing values in recipes data
recipes_df = recipes_df.dropna()

In [13]:
# Ensure the ID columns are integers
interactions_df['user_id'] = interactions_df['user_id'].astype(int)
interactions_df['recipe_id'] = interactions_df['recipe_id'].astype(int)
recipes_df['recipe_id'] = recipes_df['recipe_id'].astype(int)

In [14]:
# Display the cleaned data
print("Cleaned Interactions Data:")
print(interactions_df.head())

print("\nCleaned Recipes Data:")
print(recipes_df.head())

Cleaned Interactions Data:
   user_id  recipe_id        date  rating  \
0    38094      40893   2/17/2003       4   
1  1293707      40893  12/21/2011       5   
2     8937      44394   12/1/2002       4   
3   126440      85009   2/27/2010       5   
4    57222      85009   10/1/2011       5   

                                              review  
0  Great with a salad. Cooked on top of stove for...  
1  So simple, so delicious! Great for chilly fall...  
2  This worked very well and is EASY.  I used not...  
3  I made the Mexican topping and took it to bunk...  
4  Made the cheddar bacon topping, adding a sprin...  

Cleaned Recipes Data:
                                  recipe_name  recipe_id  minutes  \
0  arriba   baked winter squash mexican style     137739       55   
1                   all in the kitchen  chili      31490       30   
2                          alouette  potatoes     112140      130   
3          amish  tomato ketchup  for canning      59389       45   
4   

In [15]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [16]:
# Prepare the data for the surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(interactions_df[['user_id', 'recipe_id', 'rating']], reader)

In [17]:
# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

In [18]:
# Use the SVD algorithm for collaborative filtering
algo = SVD()

In [19]:
# Train the algorithm on the training set
algo.fit(trainset)

In [20]:
# Predict ratings for the test set
predictions = algo.test(testset)

In [21]:
# Function to get top N recommendations for a user
def get_top_n_recommendations(user_id, n=10):
    user_interactions = interactions_df[interactions_df['user_id'] == user_id]
    user_rated_recipes = user_interactions['recipe_id'].tolist()
    
    all_recipes = recipes_df['recipe_id'].tolist()
    unrated_recipes = [recipe for recipe in all_recipes if recipe not in user_rated_recipes]
    
    predicted_ratings = []
    for recipe_id in unrated_recipes:
        predicted_rating = algo.predict(user_id, recipe_id).est
        predicted_ratings.append((recipe_id, predicted_rating))
    
    top_n_recommendations = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:n]
    
    return [recipe_id for recipe_id, rating in top_n_recommendations]

In [22]:
# Example
user_id = 1  # Replace with a valid user ID from the interactions data
top_n = get_top_n_recommendations(user_id, n=10)
print("Top N Recommendations for User {}: {}".format(user_id, top_n))

Top N Recommendations for User 1: [232044, 137739, 31490, 112140, 59389, 44061, 25274, 67888, 75452, 109439]
